In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
Tableau10 = {'blue':'#4E79A7', 'orange':'#F28E2B', 'red':'#E15759', 'teal':'#76B7B2', 'green':'#59A14F', 
             'yellow':'#EDC948', 'purple':'#B07AA1', 'pink':'#FF9DA7', 'brown':'#9C755F', 'gray':'#BAB0AC'}

In [ ]:
# Math manipulation
import numpy as np
import pandas as pd

# Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Image processing
import cv2

# Utilities
import os
import random
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical
import codecs, json 
import re

from albumentations import Compose
import albumentations as A
import random

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

#utils
from keras_tqdm import TQDMNotebookCallback

# Models
from keras import applications
from keras.optimizers import SGD, Adam

from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import sklearn.metrics

In [ ]:
base = "/Users/megantabbutt/Desktop/Computer Science Classes/762_AdvancedDeepLearning/762_Project/"

In [ ]:
FLAG_DEBUG = False
FLAG_GENERATE_TABLE = True
FLAG_TRAIN_IN_PRELOAD = False

img_height = 256 #- these should already be done in pre-processing 
img_width = 256 #- these should already be done in pre-processing 
img_channels = 1

batch_size = 900

In [ ]:
label_names=['Human', 'Interact front', 'Interact lat', 'Interact vert', 'Crowded', 
             'Drink', 'Curiosity', 'Queue', 'Low visibility', 'Nothing']

In [ ]:
def generate_pointer_table (path_frames, path_labels): # MEGAN REFACTOR
    pt_table = []
    
    label_file = open(path_labels)
    label_data = json.load(label_file)
    
    for frame in os.listdir(path_frames):
        if frame[-4:] == ".jpg":
            key = frame.split(".")[0]
            pt_table.append([key, label_data[key]])
        
    return pt_table

In [ ]:
def do_pointer_table():
    pointer_table_path_test = path_labels_base + 'pointer_table_test.json'
    pointer_table_shuffled_path_test = path_labels_base + 'pointer_table_shuffled_test.json'
    if FLAG_DEBUG: print ('Number of files: ', len(list_of_videos))

    if FLAG_GENERATE_TABLE:      
        pt_table = generate_pointer_table(path_frames_test, path_labels)
        json.dump(pt_table, codecs.open(pointer_table_path_test, 'w', encoding='utf-8'))
        print('Test Pointer table saved')
        random.shuffle(pt_table)
        json.dump(pt_table, codecs.open(pointer_table_shuffled_path_test, 'w', encoding='utf-8'))
        print('Test Shuffled pointer table saved')
        pt_table_test = np.array(pt_table)

    test_max_index = n_test
    print(len(pt_table_test))

    gc.collect()
    
    return pt_table_test

In [ ]:
augmentation_test = A.Compose([
    A.ToFloat(max_value=255)])

In [ ]:
def frame_from_pointer (path, video, frame, verbose=False):
    if verbose: print ('In '+path+'/'+str(video)+'.mp4' + ', taking frame ' + str(frame))
    cap = cv2.VideoCapture(path+'/'+str(video)+'.mp4')
    if (cap.isOpened()== False):
        print("Error opening video file") 
        return -1
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
    ret, frame = cap.read() # Capture next frame
    if ret==True:
        cap.release()
        return frame
    else:
        print("Error opening frame") 
        return -1

In [ ]:
def frame_from_pointer(path, vidframe):
    image = cv2.imread(path + vidframe + ".jpg")
    
    if image.shape[2] > 1: # grayscale got saved as three copies of the same image in three channels so just take one TODO - MEGAN FIX
        image, G2, G3 = cv2.split(image)
        
    return image

In [ ]:
def image_generator_lazy(pt_table, batch_size, augmentation, frame_path):
    
    while True:          # Select files (paths/indices) for the batch
        pointer_batch = pt_table[np.random.choice(pt_table.shape[0], batch_size, replace=False), :]
        batch_X = []
        batch_y = []
        
        for pointer in pointer_batch:
            frame = frame_from_pointer(path=frame_path, vidframe=pointer[0])
            frame = frame.astype(np.float32) # MT added 
            #frame = augmentation(image=frame)['image']
            #print(frame)
            frame = frame.reshape(frame.shape[0], frame.shape[1], img_channels)
            label = to_categorical(pointer[1], num_classes=10)
        
            batch_X.append(frame)
            batch_y.append(label)
             
        # Return a tuple of (input,output) to feed the network          
        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        yield(batch_X, batch_y)

# Running the Data through predictions, and plotting

In [ ]:
path_frames_test = base + "Data/Exp6_frames_unbalanced_256/test/"

path_labels_base = base + "Data/Exp6_frames_unbalanced_256/labels_paper/"  
path_labels = path_labels_base + "labels_paper_test.json"

model_path = base + "ModelSave/Exp6_V1_model.h5"
acc_path = base + "ModelSave/Exp6_V1_accuracies.json"
n_test = len(os.listdir(path_frames_test))

write_data = False

#______________________________________________________________________

model = load_model(model_path) 

with open(path_labels) as fp:
    truth_data = json.load(fp)
    
data_to_write = dict()

for i, file in enumerate(os.listdir(path_frames_test)):
    if file[0] == ".":
        continue
    vid = file.split(".")[0]
    
    frame = cv2.imread(path_frames_test + file)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    down_points = (img_width, img_height)
    frame = cv2.resize(frame, down_points, interpolation=cv2.INTER_CUBIC)
    frame = np.reshape(frame, (1, 256, 256, 1))
    prediction = model.predict(frame)

    data_to_write[file] = list(prediction[0].astype(float))
    
if write_data:
    file_out = base + "Data/CNN_Predictions/"

    file_name = path_frames_test.split("/")[-3] + "_pipeline_predictions.json"

    with open(file_out + file_name, 'w') as fp:
        json.dump(data_to_write, fp)
    print("Written: ", len(data_to_write), "  to   ", file_name)
    print(file_out)

In [ ]:
# For including 0 cases only:

y_predicted = []
y_truth = []

for file in data_to_write:
    if file[0] == ".": continue
    frame = file.split(".")[0]
    #frame = file.split(".")[0].split("_")[0] + "_" + file.split(".")[0].split("_")[1]
    y_predicted.append(np.argmax(data_to_write[file]))
    
for i, file in enumerate(os.listdir(path_frames_test)):
    if file[0] == ".": continue
    info = file.split("_")
    y_truth.append(int(file.split("_")[3]))

In [ ]:
y_predicted = []
y_truth = []

for file in data_to_write:
    if file[0] == ".": continue
    frame = file.split(".")[0]
    #frame = file.split(".")[0].split("_")[0] + "_" + file.split(".")[0].split("_")[1]
    y_predicted.append(np.argmax(data_to_write[file]))
    y_truth.append(truth_data[frame])

In [ ]:
write_data = False

cm = confusion_matrix(y_truth, y_predicted)

FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

F1 = 2*TP / (2*TP + FP + FN)
PREC = TP / (TP + FP)
REC = TP / (TP + FN)

stats_to_write = dict()
stats_to_write["FP"] = list(FP.astype(float))
stats_to_write["FN"] = list(FN.astype(float))
stats_to_write["TP"] = list(TP.astype(float))
stats_to_write["TN"] = list(TN.astype(float))

if write_data:
    file_out = base + "Data/CNN_Predictions/"

    file_name = path_frames_test.split("/")[-3] + "_stats.json"

    with open(file_out + file_name, 'w') as fp:
        json.dump(stats_to_write, fp)
    print("Written: ", len(stats_to_write), "  to   ", file_name)
    print(file_out)

print(FP)
print(FN)
print(TP)
print(TN)

In [ ]:
print("precision, recall, Fscore, support")
sklearn.metrics.precision_recall_fscore_support(y_truth, y_predicted, average='weighted')

In [ ]:
corr = 0 
for i in range(cm.shape[0]):
    corr += cm[i, i]
 
print("Overall Accuracy:")
corr/ np.sum(np.sum(cm, axis=1, keepdims=True))*100

In [ ]:
#exp = np.zeros(cm.shape)
#for i in range(cm.shape[0]):
#    exp[i, i] = (cm[i, i] / np.sum(cm, axis=1, keepdims=True)[i]) * np.max((cm * (np.ones(cm.shape) - np.identity(cm.shape[0]))))
#cm_norm = exp - (cm * (np.ones(cm.shape) - np.identity(cm.shape[0])))

In [ ]:
cm_norm = np.zeros(cm.shape)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        cm_norm[i, j] = cm[i, j] / np.sum(cm, axis=1, keepdims=True)[i]

In [ ]:
write_data = True

def plot_cm(labels_x, labels_y, title):
    fig, ax = plt.subplots(figsize=(10, 10))
    cax = ax.matshow(cm_norm, cmap=plt.cm.YlGnBu, alpha=.9, vmin=0, vmax=1.)
    cbar = fig.colorbar(cax, fraction=0.045)
    cbar.set_label(label='Occurance', size=18)
    cbar.outline.set_edgecolor('white')
    cbar.set_ticks([])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            cm_sum = np.sum(cm, axis=1, keepdims=True)
            cm_perc = cm / cm_sum.astype(float)
            color = 'black'
            if cm_norm[i, j] > .50: color = 'white'
            if i == j:
                ax.text(x=j, y=i-.2, s="{:.1%}".format(cm_perc[i, j]), va='center', ha='center', size='large', c=color, fontweight='bold')
                ax.text(x=j, y=i+.2, s="{}/{}".format(cm[i, j], int(cm_sum[i])), va='center', ha='center', size='large', c=color)
            else:
                ax.text(x=j, y=i,s="{}".format(cm[i, j]), c=color, va='center', ha='center', size='medium')

    ax.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
    ax.set_xticklabels(labels=labels_x, rotation=30, fontsize='large')
    ax.set_yticklabels(labels=labels_y, fontsize='large')
    ax.spines['top'].set_visible(False), ax.spines['right'].set_visible(False), 
    ax.spines['bottom'].set_visible(False), ax.spines['left'].set_visible(False)
    
    plt.xlabel('Predicted', fontsize=18)
    plt.ylabel('Actual', fontsize=18, )
    plt.title(title, y=1.02, fontsize=18)
    fig.tight_layout()
    if write_data:
        file_out = base + "Data/CNN_Predictions/"
        file_name = path_frames_test.split("/")[-3] + "_confusionMatrix"
        plt.savefig(file_out+file_name, dpi=550, bbox_inches='tight')
    plt.show()
    
plot_cm(['', 'Group Interact.', 'Drinking', 'Waiting in Line'], 
       ['', 'Group Interact.', 'Drinking', 'Waiting in Line'], 
       title="Exp4: Cropped Images")

# ['', 'Nothing', 'Group Interact.', 'Drinking', 'Waiting in Line']
# ['', 'Group Interact.', 'Drinking', 'Waiting in Line']
# ['', 'Interact front', 'Interact lat', 'Interact vert', 'Crowded', 'Drink', 'Curiosity', 'Queue', 'Low visibility', 'Nothing']

In [ ]:
write_data = True

with open(acc_path) as fp:
    accuricies = json.load(fp)
    
acc = accuricies['acc']
val_acc = accuricies['val_acc']
loss = accuricies['loss']
val_loss = accuricies['val_loss']
epochs = range(1, len(acc) + 1)

fig, axs = plt.subplots(figsize=(6, 4))
axs.spines['top'].set_visible(False), axs.spines['right'].set_visible(False), 
axs.spines['bottom'].set_visible(True), axs.spines['left'].set_visible(True)

axs.plot(epochs, acc, c=Tableau10['blue'], label='Training')
axs.plot(epochs, val_acc, c=Tableau10['red'], label='Validation')
axs.set_xlabel("Epochs", fontsize=18)
axs.set_ylabel("Accurarcy", fontsize=18)
axs.grid()

axs.legend(loc="lower right", fontsize=14)
plt.title("Exp6: Cropped Images - Weak Supervision", fontsize=16)
fig.tight_layout()

if write_data:
    file_out = base + "Data/CNN_Predictions/"
    file_name = path_frames_test.split("/")[-3] + "_accuracies"
    plt.savefig(file_out+file_name, dpi=550, bbox_inches='tight')
plt.show()

In [ ]:
#classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#for cla in classes:
#    correct = 0
#    total = 0
#    for key in data_to_write:
#        vid_name = key.split(".")[0]
#       if truth_data[vid_name] == cla:
#            total += 1
#            if truth_data[vid_name] == np.argmax(data_to_write[key]):
#                correct += 1
#                
#    if total > 0:
#        print(cla, total, correct/total)
#    else:
#        print(cla)

# Prepping some stats info:

In [ ]:
base = "/Users/megantabbutt/Desktop/Computer Science Classes/762_AdvancedDeepLearning/762_Project/Data/CNN_Predictions/"
file_in = "Exp3_frames_unbalanced_256_manual_A_stats.json"

with open(base + file_in) as fp:
    stats = json.load(fp)
    
def get_acc_stats(FP, FN, TP, TN):
    
    if (TP + TN + FP + FN) == 0: ACC = np.nan
    else: ACC = (TP + TN) / (TP + TN + FP + FN)
    
    if (2*TP + FP + FN) == 0: F1 = np.nan
    else: F1 = 2*TP / (2*TP + FP + FN)
    
    if (TP + FP) == 0: PREC = np.nan
    else: PREC = TP / (TP + FP)
    
    if (TP + FN) == 0: REC = np.nan
    else: REC = TP / (TP + FN)
    
    return ACC, F1, PREC, REC

In [ ]:
for i in range(len(stats['FP'])):
    ACC, F1, PREC, REC = get_acc_stats(stats['FP'][i], stats['FN'][i], stats['TP'][i], stats['TN'][i])
    print("{:.1%},\t {:.1%},\t {:.1%},\t {:.1%}".format(ACC, F1, PREC, REC))

In [ ]:
stats